### IMPORTANT:
Before running the code do the following steps:

1) Create an anaconda environment (python), and install the packages skillbridge and jupyter notebook.
2) Make sure the terminal used for this notebook and for Cadence Virtuoso are both on the same environment
3) Create a blank layout cell in Cadence in the desired process
5) Navigate to the layout window in Cadence
6) Type "skillbridge path" into the terminal in the conda environment. Type the resulting "load()" statement into the Cadence CIW window
7) Type pyStartServer ?id "[your-server-id]" into the Cadence CIW window. This will set up a server to link python to Cadence through skillbridge using the server id you choose
8) open the EMX gui and set the following settings before continuing. 

    EMX:
    Process: 9HP_2023_Process/422OLLD.proc
    
    Accuracy options:
    	- divides the design into finite elements
    		1, 1, 0.5, *
    		for these options
    Ports:
    	signals: LEFT RIGHT BOTTOM TOP LEFT_m RIGHT_m BOTTOM_m TOP_m
    
    Start:
    	30e9
    Stop
    	100e9
    Step
    	1e9
    
    Advanced Options:
    	- make Label depth = 0 
    	- cadence pins checked
9) Set the following variables in the next code block:
10) EMX_WINDOW_NUMBER: the next window number that emx results will pop up to. You can find this by closing the most recent run and checking what number it was in the Virtuoso CIW window. Add one to this.
11) trials: number of simulations to run
12) start_num: number to start at for labeling files in results directory
13) EMX_WORK_PATH: the path to the EMX_work directory, which is where emx stores results on your machine
14) RESULTS_DIR: the path to the directory where you would like to store the resulting data.



In [ ]:
import time, os, shutil
from EMXSimulator3 import EMXSimulator as sim

# figure out which window number the emx window will pop up as in order to close after running a test
EMX_WINDOW_NUMBER=11945
# starts at 3 in new cadence instance

# Run 1000 emx simulations, deleting and reinitializing a random structure each time
trials = 10000
# for naming files, make sure to update to start at most recent
# i.e. if there are 10000 data points in the results already, start at 10000
start_num = 10000

# desired structure size (roughly the length of square side) in microns
STRUCT_SIZE = (300, 300)

# name of layout cell
LIBRARY_NAME = "Thesis"
CELL_NAME = "test3"

# EMX work folder path
EMX_WORK_PATH = "/home/sbfisher/Cadence/9HPsetup/EMX_work/"

# directory to store results
RESULTS_DIR = "/scratch/gpfs/sbfisher/Test_Results300/"

# static left, right, bottom, top pins
# STATIC_PINS = [(8,0),(8,17),(17,7),(0,7)]

# Initialize Simulator
Simulator = sim("server-3", struct_width=STRUCT_SIZE[0], struct_height=STRUCT_SIZE[1])
# create first random structure
Simulator.create_random_structure()
# Create pins
Simulator.create_pins()
# Simulator.create_pins(pin_locations=[(8,0),(8,17),(17,7),(0,7)])

for i in range(trials):
    # delete current sparam file if it exists
    sparam_file = EMX_WORK_PATH + "{}_{}.work/{}.s8p".format(LIBRARY_NAME, CELL_NAME, CELL_NAME)
    if os.path.exists(sparam_file):
        os.remove(sparam_file)
    
    print("Starting Simulation #{}. ".format(i))
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    
    # run emx, creates sparam file
    Simulator.run_emx()

    # wait for s_param_file to be 1139 lines (kinda hacky, but this ensures that the simulation has completed
    line_count = 0
    while line_count < 1139:
        time.sleep(1) # give emx time to create sparam file. Might be better way if this ends up failing
        try:
            s_param_file = open(sparam_file, "r")
            lines = s_param_file.readlines()
            line_count = len(lines)
            s_param_file.close()
        except FileNotFoundError:
            print("s param file not created yet. Failing gracefully")
        # else:
        #     print("other error when reading sparam file")

    
    # move completed sparam file to data folder
    shutil.move(sparam_file, RESULTS_DIR + "trial_{}_data.s8p".format(i+start_num));
     # Create a file with the form of the structure and store
    Simulator.create_struct_file(RESULTS_DIR + "trial_{}_struct.txt".format(i+start_num))

    # delete current random structre and create new one
    Simulator.delete_structure()
    Simulator.delete_pins()
    Simulator.create_random_structure()
    Simulator.create_pins()

    # Simulator.create_pins(pin_locations=[(8,0),(8,17),(17,7),(0,7)])

    # Close EMX sim window
    Simulator.close_window(EMX_WINDOW_NUMBER+i)
    
    time.sleep(0.5) # try to fix job currently running error
    # Figure out which window number you are currently on for this
    print("Completed Simulation #{}".format(i))
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    print()

Simulator.delete_structure()
Simulator.delete_pins()
Simulator.delete_rect()

Starting Simulation #0. 
16:53:43
Completed Simulation #0
16:53:54

Starting Simulation #1. 
16:53:54
Completed Simulation #1
16:54:05

Starting Simulation #2. 
16:54:05
Completed Simulation #2
16:54:15

Starting Simulation #3. 
16:54:15
Completed Simulation #3
16:54:25

Starting Simulation #4. 
16:54:25
Completed Simulation #4
16:54:35

Starting Simulation #5. 
16:54:35
Completed Simulation #5
16:54:45

Starting Simulation #6. 
16:54:45
Completed Simulation #6
16:54:55

Starting Simulation #7. 
16:54:55
Completed Simulation #7
16:55:05

Starting Simulation #8. 
16:55:05
Completed Simulation #8
16:55:17

Starting Simulation #9. 
16:55:17
Completed Simulation #9
16:55:29

Starting Simulation #10. 
16:55:29
Completed Simulation #10
16:55:38

Starting Simulation #11. 
16:55:38
Completed Simulation #11
16:55:49

Starting Simulation #12. 
16:55:49
Completed Simulation #12
16:55:58

Starting Simulation #13. 
16:55:58
Completed Simulation #13
16:56:12

Starting Simulation #14. 
16:56:12
Compl